In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

In [2]:
#Veri setinin eklenip başlığının belirlenmesi
column = ['yorum']
df = pd.read_csv('yorumlar.csv', encoding ='iso-8859-9', sep='"')
df.columns=column
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19999 entries, 1 to 19999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   yorum   19999 non-null  object
dtypes: object(1)
memory usage: 312.5+ KB


,yorum
1,Bu filmin katıldığı festivaller ödüllerini fi...
2,çok komik bir film ya izlediğim en iyi komedi...
3,harbiden çooooooook iyiydi herkesin dediği gi...
4,hayaller çok geniştir ve insanlar hayallerini...
5,o kadar sıcak ve samimi bir filmki tebrik etm...


In [3]:
#Veri setindeki Türkçe dolgu kelimelerinin kaldırılması
def remove_stopwords(df_fon):
    stopwords = open('turkce-stop-words', 'r').read().split()
    df_fon['stopwords_removed'] = list(map(lambda doc:
        [word for word in doc if word not in stopwords], df_fon['yorum']))
remove_stopwords(df)

In [4]:
#Verisetinde Positivity adlı bir sütunun oluşturalım ve başlangıçta tüm değerlere olumlu olarak 1 değerinin atayalım
df['Positivity'] = 1

In [6]:
#Veri setimizde 10003. veri ve sonrasındaki veriler olumsuz(negatif) verilerdir bu yüzden bu değerlerin
#Positivity değerleri 0 ile değiştirelim.
df.Positivity.iloc[10003:] = 0

In [11]:
#Şimdi, verileri "yorum" ve "Positivity" sütunlarını kullanarak rastgele eğitim ve test alt kümelerini bölüştürelim ve 
#ardından ilk girişi ve eğitim setinin şeklini yazalım.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['yorum'], df['Positivity'], random_state = 0)
print(X_train.tail())
print('\n\nX_train shape: ', X_train.shape)

13124     Film zaman geçirmek için iyi sayılabilir ama ...
19649     kalitesi gerçekten düşük bir animasyon ayrıca...
9846      filmi daha 2. gününde izledim ama yorum yapma...
10800     şener şen'in oyunculuğu dışında hiç güzel değ...
2733       Hayatımdaki yeri bambaşka olan bir film. 10/10 
Name: yorum, dtype: object


X_train shape:  (14999,)


In [13]:
#CountVectorizer'ı başlatıyoruz ve eğitim verilerimize uyguluyoruz.
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(encoding ='iso-8859-9').fit(X_train)

#X_train'deki belgeleri bir belge terim matrisine dönüştürürüz
X_train_vectorized = vect.transform(X_train) 

#Bu özellik matrisi X_ train_ vectorized'e dayanarak Lojistik Regresyon sınıflandırıcısını eğiteceğiz
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

C:\Users\betul\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
#Modelimizin bu tahminleri nasıl yaptığını daha iyi anlamak için, her bir özellik için katsayıları (bir kelime), 
#pozitifliği ve olumsuzluk açısından ağırlığını belirlemek için kullanabiliriz.
feature_names = np.array(vect.get_feature_names())
sorted_coef_index = model.coef_[0].argsort()
print('Negatif: \n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Pozitif: \n{}\n'.format(feature_names[sorted_coef_index[:-11:-1]]))


Negatif: 
['berbat' 'beğenmedim' 'sıkıcı' 'rezalet' 'vasat' 'değmez' 'gereksiz'
 'sıkıldım' 'kötü' 'etmiyorum']

Pozitif: 
['beğendim' 'mükemmel' 'müthiş' 'harika' 'söze' 'başyapıt' 'izlenmeli'
 'zleyin' 'muhteşem' 'budur']



In [15]:
#tf-idf vectorizer'ı başlatacağız ve eğitim verilerimize uygulayacağız. 
#En az beş dokümanda görünen kelime dağarcığımızdaki kelimeleri kaldıracağımız min_df = 5 değerini belirtiyoruz.
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(min_df = 5).fit(X_train)

X_train_vectorized = vect.transform(X_train)
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)
predictions = model.predict(vect.transform(X_test))
a=roc_auc_score(y_test, predictions)
print('AUC: ', a)  

feature_names = np.array(vect.get_feature_names())
sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
print('En küçük Tfidf: \n{}\n'.format(feature_names[sorted_tfidf_index[:10]]))
print('En büyük Tfidf: \n{}\n'.format(feature_names[sorted_tfidf_index[:-11:-1]]))


AUC:  0.8789689969970661
En küçük Tfidf: 
['tazegül' 'durumla' 'isi' 'report' 'dönmesi' 'düşünme' 'öfke'
 'karşımızdaki' 'oturan' 'kuzey']

En büyük Tfidf: 
['basit' 'ben' 'etkileyici' 'vasat' 'cok' 'yorumsuz' 'izleyin' 'bence'
 'erkek' 'sevmedim']



In [16]:
#bigramlar, bitişik kelimelerin çiftlerini sayar ve bize kötü ve kötü olmayan gibi özellikler verebilir. 
#Bu nedenle, minimum 5 belge sıklığını belirten ve 1 gram ve 2 gram ekstrakt eden eğitim setimizi yeniden yerleştiriyoruz.
vect = CountVectorizer(min_df = 5, ngram_range = (1,2)).fit(X_train)
X_train_vectorized = vect.transform(X_train)

model = LogisticRegression()
model.fit(X_train_vectorized, y_train)
predictions = model.predict(vect.transform(X_test))
print('AUC: ', roc_auc_score(y_test, predictions))

C:\Users\betul\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


AUC:  0.8761598679758971


In [ ]:
#Her özelliği kontrol etmek için katsayıları kullanarak görebiliriz
feature_names = np.array(vect.get_feature_names())
sorted_coef_index = model.coef_[0].argsort()
print('Negatif: \n{}\n'.format(feature_names[sorted_coef_index][:10]))
print('Pozitif: \n{}\n'.format(feature_names[sorted_coef_index][:-11:-1]))
while(True):
    yorum=input("Yorumunuz Nedir?(Programdan çıkmak için \'F\' yazınız)")
    if(yorum == 'F' or yorum == 'f'):
        break
    else:
        print(model.predict(vect.transform([yorum])))

Negatif: 
['berbat' 'beğenmedim' 'rezalet' 'sıkıcı' 'en kötü' 'değmez' 'mahsun'
 'gereksiz' 'vasat' 'yahudi']

Pozitif: 
['10 10' 'mükemmel' 'beğendim' 'müthiş' 'filme kötü' 'harika' 'izlenmeli'
 'rocky' 'başyapıt' 'budur']

Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)güzel
[1]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)güzel
[1]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)berbat
[0]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)güzel değil
[0]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)musluk
[1]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)harika
[1]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)güzel
[1]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)mükemmel
[1]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)dolap
[1]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)rezalet
[0]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)kötü
[0]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)iyi değil
[0]
